In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/nemotron

Mounted at /content/drive
/content/drive/MyDrive/nemotron


In [ ]:
!pip install "torch==2.5.0"

In [ ]:
!pip install -qU transformers accelerate bitsandbytes peft trl datasets evaluate #ai2-olmo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd


data_type='Original'
#data_type='Synthetic'

# Load Data
df = pd.read_excel('LR_Dataset_Original_Sythetic_Final.xlsx')
#df = pd.read_excel('LR_Dataset_Original_Sythetic_Experiment_70.xlsx') # 20% of the training and validation sets


if data_type=='Original':
 df = df[df['Source'] == 'Original'] # Only original

df= df[['Sentence','Category' ,'Classification']]

df['Sentence'] = df['Sentence'].str.capitalize()
df=df.sample(frac=1).reset_index(drop=True)
df['Sentence'] = (df['Sentence']
                  .str.strip()
                  .str.replace(r'\n|\r', ' ', regex=True)
                  .str.replace(r'\s{2,}', ' ', regex=True))
df

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-878308702a4f>", line 1, in <cell line: 1>
    import pandas as pd
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback


In [ ]:
def create_instruction(row):
    sentence = row['Sentence']
    instruction = (
                "You are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\n"
                "Categories:\n "
                "1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n "
                "2. RESEARCH GAP: Highlights the need for further research within the topic.\n"
                "3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n"
                "4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “showed”, “found”, “demonstrated”, and “observed” or phrases like “the findings indicate”.\n"
                "5. LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study.\n"
                "6. EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the words “we” or “our”.\n"
                "7. OTHER: Any text that does not fit the above categories.\n"
                "Procedure:\n"
                "1. Determine whether the subject of the sentence is a topic or a study.\n"
                "2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\n"
                "3. Provide the category number that best fits the sentence. Just provide the category number without any explanation.\n"

                f"Sentence: {sentence}.\n"
            )

    return instruction

In [ ]:
df['instruction'] = df.apply(create_instruction, axis=1)

df = df.rename(columns={'Category': 'response'})

df

,Sentence,response,Classification,instruction
0,"Powered by gnn, many inductive embedding metho...",OVERALL,TEST,You are a researcher that should assign a clas...
1,Many online games feature multiple ways to soc...,OVERALL,VALIDATION,You are a researcher that should assign a clas...
2,Visual techniques have been used to show high-...,OVERALL,TRAINING,You are a researcher that should assign a clas...
3,"In the same vein, rico and brewster showed tha...",RESULT,TRAINING,You are a researcher that should assign a clas...
4,"Ttranse [23], the variant of transe [4], incor...",DESCRIPTION,VALIDATION,You are a researcher that should assign a clas...
...,...,...,...,...
695,"Advances in ml, largely in language models, ha...",OVERALL,TEST,You are a researcher that should assign a clas...
696,A recent study found similar results that huma...,RESULT,TRAINING,You are a researcher that should assign a clas...
697,Knowledge graph entity typing (kget) is an ess...,OVERALL,TRAINING,You are a researcher that should assign a clas...
698,Another major challenge is to find an efficien...,RESEARCH GAP,TRAINING,You are a researcher that should assign a clas...


In [ ]:
test_dataset =df[df['Classification'] == 'TEST']
test_dataset

,Sentence,response,Classification,instruction
0,"Powered by gnn, many inductive embedding metho...",OVERALL,TEST,You are a researcher that should assign a clas...
10,Although not explicitly mentioning double-coun...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
16,"Thus, our approach does not sit cleanly in the...",EXTENSION,TEST,You are a researcher that should assign a clas...
20,Tie [41] is a time-aware incremental embedding...,DESCRIPTION,TEST,You are a researcher that should assign a clas...
29,There is lacking guidance for many of the pris...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...
...,...,...,...,...
667,"Recently, large language models have shown the...",OVERALL,TEST,You are a researcher that should assign a clas...
670,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...
677,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...
686,Our encoding of traces does allow for uneven d...,LIMITATION,TEST,You are a researcher that should assign a clas...


In [ ]:
def generate_test_prompt(example):
    """Format prompt for training."""
    text = f"<|im_start|>user\n{example['instruction']}<|im_end|>"
    return text

In [ ]:
test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


<ipython-input-9-ca85691956fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['text']=test_dataset.apply(generate_test_prompt, axis=1)


In [ ]:
test_dataset

,Sentence,response,Classification,instruction,text
0,"Powered by gnn, many inductive embedding metho...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
10,Although not explicitly mentioning double-coun...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
16,"Thus, our approach does not sit cleanly in the...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
20,Tie [41] is a time-aware incremental embedding...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
29,There is lacking guidance for many of the pris...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
...,...,...,...,...,...
667,"Recently, large language models have shown the...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
670,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
677,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...
686,Our encoding of traces does allow for uneven d...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

# Model Setup (Quantization and Tokenizer)

model_name="mgoin/nemotron-3-8b-chat-4k-sft-hf"



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    #bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)


base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True,
    #torch_dtype=torch.float16, # This new #torch.bfloat16
)

# Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(model_name)
                                          #torch_dtype=torch.float16, # This new #torch.bfloat16
                                          #device_map="auto", # Added
                                          #trust_remote_code=True,)



tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.1G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.1G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "Nemotron_8B_Instruct-lro-finetune/checkpoint-245")

In [ ]:
def extract_text(text):
    # Define both markers to search for
    markers = ["<|im_end|>assistant", "assistant\n"]

    # Loop through markers and check if each is in the text
    for marker in markers:
        marker_position = text.find(marker)

        # If the marker is found, extract text after it
        if marker_position != -1:
            return text[marker_position + len(marker):].strip()  # Remove any leading/trailing whitespace

    # Return None if neither marker is found
    return text

In [ ]:
def clean_text(text):

    # Check for keywords and replace text accordingly
    if "OVERALL" in text:
        return "OVERALL"
    if "RESEARCH GAP" in text:
        return "RESEARCH GAP"
    if "DESCRIPTION" in text:
        return "DESCRIPTION"
    if "RESULT" in text:
        return "RESULT"
    if "LIMITATION" in text:
        return "LIMITATION"
    if "EXTENSION" in text:
        return "EXTENSION"
    elif "OTHER" in text:
        return "OTHER"

    # If neither keyword is found, return the original text
    return text

In [ ]:
prompt=test_dataset.iloc[0]['text']
prompt


'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
inputs = tokenizer(prompt,
                return_tensors="pt"
            ).to("cuda")

outputs = ft_model.generate(**inputs, max_new_tokens=15, use_cache=True)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


'<|im_start|>user\nYou are a researcher that should assign a classification to a sentence from scientific articles, choosing from one of the following seven categories. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\nCategories:\n 1. OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together.\n 2. RESEARCH GAP: Highlights the need for further research within the topic.\n3. DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.\n4. RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like “sh

In [ ]:
clean_text(extract_text(answer))

'OVERALL'

In [ ]:
def get_classification_finetuning(data_point,model,tokenizer):
    """
    Gets the classification for a data point using the fine-tuned model.
    """
    inputs = tokenizer(data_point['text'],return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=15, use_cache=True)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    print(clean_text(extract_text(answer)))
    data_point['Prediction_Finetune']=answer  # Assign the result to the data point
    data_point['Prediction_Finetune_Cleaned']=clean_text(extract_text(answer))

    return data_point
# Apply the get_classification function to the dataset using map
test_dataset = test_dataset.apply(lambda row: get_classification_finetuning(row, ft_model, tokenizer), axis=1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OVERALL


/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


DESCRIPTION


/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

EXTENSION


/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


LIMITATION


/usr/local/lib/python3.10/dist-packages/transformers/models/nemotron/modeling_nemotron.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


OVERALL


In [ ]:
test_dataset


,Sentence,response,Classification,instruction,text,Prediction_Finetune,Prediction_Finetune_Cleaned
0,"Powered by gnn, many inductive embedding metho...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,OVERALL
10,Although not explicitly mentioning double-coun...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,DESCRIPTION
16,"Thus, our approach does not sit cleanly in the...",EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,EXTENSION
20,Tie [41] is a time-aware incremental embedding...,DESCRIPTION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,DESCRIPTION
29,There is lacking guidance for many of the pris...,RESEARCH GAP,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,RESEARCH GAP
...,...,...,...,...,...,...,...
667,"Recently, large language models have shown the...",OVERALL,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,OVERALL
670,Our work builds on and generalises the promine...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,EXTENSION
677,To improve the ability of preference-extractor...,EXTENSION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,EXTENSION
686,Our encoding of traces does allow for uneven d...,LIMITATION,TEST,You are a researcher that should assign a clas...,<|im_start|>user\nYou are a researcher that sh...,<|im_start|>user\nYou are a researcher that sh...,LIMITATION


In [ ]:
test_dataset.to_csv('Nemotron8B_FT_Test_LoRA_Augmented.csv', index=False)
#test_dataset.to_csv('SmolLM2_FT_Test_NEFT_Augmented.csv', index=False)
